## Sentinel-1 offset tracking

This application takes a pair of Sentinel-1 products and identifies and generates the coherence

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

As a data processor developer, I want to implement, and package an algorithm processing a pair of S1 SAR SLC datasets using the SNAP toolbox notebook archetype with the following processing steps:

A. S1 SAR processing per image:

* Application of orbit file (should wait for the orbit file a couple of days, since for coherence this is important)
* TOPS slice assembly (if necessary)
* TOPS split (if necessary)

B. For image pair:

* TOPS coregistration
* Coherence estimation (Given set of images from same orbit,  {t1, t2, t3, ... , tn}, two temporally adjacent images would * constitute an image pair, with the first one being the master. So the pairs would be: {t1 - t2, t2 - t3, ... , tn-1 - tn}.)
* TOPS deburst
* Multi-looking
* Terrain correction


TOPS --> Terrain Observation by Prograssive Scans

TOPS Slice assembly-->assemble subswaths back together

TOPS split -->split into subswaths



### <a name="service">Service definition

In [4]:
service = dict([('title', 'Sentinel-1 offset tracking'),
                ('abstract', 'Sentinel-1 offset tracking'),
                ('id', 'offset-tracking')])

In [29]:
max_velocity = dict([('id', 'max_velocity'),
               ('value', '4'),
               ('title', 'Max velocity'),
               ('abstract', 'Max velocity (m/day)')])

In [8]:
polarisation = dict([('id', 'polarisation'),
                     ('value', 'VV'),
                     ('title', 'Polarisation'),
                     ('abstract', 'Polarisation')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

These are the Sentinel-1 product identifiers

In [9]:
input_identifiers = ('S1A_IW_SLC__1SDV_20181005T214303_20181005T214335_024006_029F71_C8CD',
                     'S1A_IW_SLC__1SDV_20181005T214237_20181005T214305_024006_029F71_5227',
                     'S1A_IW_SLC__1SDV_20180607T214217_20180607T214244_022256_026888_3B32',
                     'S1A_IW_SLC__1SDV_20180607T214242_20180607T214309_022256_026888_17C7')

**Input references**

These are the Sentinel-1 catalogue references

In [10]:
input_references = ('https://catalog.terradue.com/sentinel1/search?format=json&uid=S1A_IW_SLC__1SDV_20181005T214303_20181005T214335_024006_029F71_C8CD',  
                    'https://catalog.terradue.com/sentinel1/search?format=json&uid=S1A_IW_SLC__1SDV_20181005T214237_20181005T214305_024006_029F71_5227',        
                    'https://catalog.terradue.com/sentinel1/search?format=json&uid=S1A_IW_SLC__1SDV_20180607T214217_20180607T214244_022256_026888_3B32',            
                    'https://catalog.terradue.com/sentinel1/search?format=json&uid=S1A_IW_SLC__1SDV_20180607T214242_20180607T214309_022256_026888_17C7')

**Data path**

This path defines where the data is staged-in. 

In [11]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [13]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())
import ellip_snap_helpers

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
from datetime import datetime

import gzip
import shutil

import gdal
import osr

import lxml.etree as etree

from shapely.wkt import loads
import numpy as np

from shapely.geometry import box

import warnings
warnings.filterwarnings("ignore")

import glob

sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors



import cioppy
ciop = cioppy.Cioppy()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read the products

### check if all the products have the same track number

In [33]:
product_TR = [None]*len(input_references)

for index,product_ref in enumerate(input_references):
    
    result_prod = ciop.search(end_point=product_ref,
                              params=[],
                              output_fields='startdate,track',
                              model='EOP')
    
    product_TR[index] = result_prod[0]['track']
    
    if index==0:
        
        slave_date = result_prod[0]['startdate'][:10]
    
    elif result_prod[0]['startdate'][:10] > slave_date:
    
        slave_date = result_prod[0]['startdate'][:10]

if not all(x == product_TR[0] for x in product_TR):

    raise ValueError('Not all products pertain the same track !')

In [35]:
slave_date

'2018-10-05'

#### Read the products

In [17]:
s1meta = "manifest.safe"

slave_date = ''

slave_products = []
master_products = []

slave_prefix = []
master_prefix = []

dates = []

for index, identifier in enumerate(input_identifiers):

    
    s1_zip_file = os.path.join(data_path, identifier + '.zip') 
    s1_meta_file = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe') 

    if os.path.isfile(s1_zip_file):
        s1prd = s1_zip_file
    elif os.path.isfile(s1_meta_file):
        s1prd = s1_meta_file

    #print identifier, s1prd
    reader = ProductIO.getProductReader("SENTINEL-1")
    product = reader.readProductNodes(s1prd, None)
    
    
    width = product.getSceneRasterWidth()
    height = product.getSceneRasterHeight()
    name = product.getName()
    start_date = parser.parse(product.getStartTime().toString()).isoformat()
    
    dates.append(start_date[:19])

    if start_date[:10] == slave_date:
        
            slave_products.append(s1prd)
            print("\nProduct: %s, %d x %d pixels of %s assigned as slave" % (name, width, height, start_date))
            slave_prefix.append(identifier.split('_')[-1]) 
            slave_data_take = identifier.split('_')[-2]
    else:
            master_products.append(s1prd)
            print("\nProduct: %s, %d x %d pixels of %s assigned as master" % (name, width, height, start_date))
            master_data_take = identifier.split('_')[-2]  
            master_prefix.append(identifier.split('_')[-1]) 

            
            
output_name = 'S1_OFFSET_TRACKING_%s_%s_%s_%s_%s_%s_%s_%s' % (parser.parse(min(dates)).strftime('%Y%m%d%H%M%S'),
                                                     parser.parse(max(dates)).strftime('%Y%m%d%H%M%S'),
                                                     slave_data_take,
                                                     len(input_identifiers)/2, 
                                                     '_'.join(slave_prefix),
                                                     master_data_take,
                                                     len(input_identifiers)/2, 
                                                     '_'.join(master_prefix))

print("\nco-registered OUTPUT Img name is %s"%output_name)


NameError: name 's1prd' is not defined

In [37]:
mygraph = ellip_snap_helpers.GraphProcessor()

#### Read and if need assemble the products

In [12]:
operator = 'Read'

node_id = 'Read'

source_node_id = ''

In [13]:
if len(slave_products) > 1:
  
    slave_read_nodes = []
    
    # Read 
    for index, slave_identifier in enumerate(slave_products):
        
        operator = 'Read'
        
        parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
        node_id = 'Read-S(%s)' % index
        
        source_node_id = ''
        
        parameters['file'] = slave_identifier
                
        mygraph.add_node(node_id, operator, parameters, source_node_id)
    
        slave_read_nodes.append(node_id)
    
    
    source_nodes_id = slave_read_nodes
        
    operator = 'SliceAssembly'
       
    node_id = 'SliceAssembly-S'
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    parameters['selectedPolarisations'] = polarisation['value']
    
    mygraph.add_node(node_id, operator, parameters, source_nodes_id)

    source_slave_orbit = node_id
    
else:
    
    operator = 'Read'
        
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
    node_id = 'Read-S'
        
    source_node_id = ''
        
    parameters['file'] = slave_products[0]
        
    mygraph.add_node(node_id, operator, parameters, source_node_id)
    
source_slave_orbit = node_id

In [14]:
if len(master_products) > 1:
  
    master_read_nodes = []
    
    # Read 
    for index, master_identifer in enumerate(master_products):
        
        operator = 'Read'
        
        parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
        node_id = 'Read-M(%s)' % index
        
        source_node_id = ''
        
        parameters['file'] = master_identifer
        
        mygraph.add_node(node_id, operator, parameters, source_node_id)
    
        master_read_nodes.append(node_id)
    
    
    source_nodes_id = master_read_nodes
        
    operator = 'SliceAssembly'
       
    node_id = 'SliceAssembly-M'
    
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
    
    parameters['selectedPolarisations'] = polarisation['value']
    
    mygraph.add_node(node_id, operator, parameters, source_nodes_id)

    source_master_orbit = node_id
    
else:
    
    operator = 'Read'
        
    parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
        
    node_id = 'Read-M'
        
    source_node_id = ''
        
    parameters['file'] = master_products[0]
        
    mygraph.add_node(node_id, operator, parameters, source_node_id)
    
source_master_orbit = node_id

In [38]:
mygraph.view_graph()


<graph>
  <version>1.0</version>
</graph>



### Apply orbit file

In [16]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File-S' 

source_node_id = source_slave_orbit

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

In [17]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [18]:
operator = 'Apply-Orbit-File'

node_id = 'Apply-Orbit-File-M' 

source_node_id = source_master_orbit

In [19]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

In [20]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read-S(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_SLC__1SDV_20170105T035027_20170105T035041_014691_017E74_5F66/S1A_IW_SLC__1SDV_20170105T035027_20170105T035041_014691_017E74_5F66.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Read-S(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110/S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="SliceAssembly-S">
    <operator>SliceAssembly</operator>
    <sources>
      <sourceProduct refid="Read-S(0)"/>
      <sourceProduct.1 refid="Read-S(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding

### DEM assisted coregistration

In [18]:
operator = 'DEM-Assisted-Coregistration'

node_id = 'DEM-Assisted-Coregistration' 

source_node_id = ['Apply-Orbit-File-S',
                 'Apply-Orbit-File-M']

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

In [20]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

NameError: name 'mygraph' is not defined

In [30]:
operator = 'Offset-tracking'

node_id = 'Offset-tracking' 

source_node_id = 'DEM-Assisted-Coregistration' 

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)

In [31]:
parameters['maxVelocity'] = max_velocity['value']

In [32]:
parameters

{'averageBoxSize': '5',
 'fillHoles': 'true',
 'gridAzimuthSpacing': '40',
 'gridRangeSpacing': '40',
 'maxVelocity': '4',
 'radius': '4',
 'registrationWindowHeight': '128',
 'registrationWindowWidth': '128',
 'resamplingType': 'BICUBIC_INTERPOLATION',
 'roiVector': None,
 'spatialAverage': 'true',
 'xCorrThreshold': '0.1'}

In [22]:
mygraph.add_node(node_id, operator, parameters, source_node_id)

NameError: name 'mygraph' is not defined

In [ ]:
operator = 'Write'

parameters = ellip_snap_helpers.get_operator_default_parameters(operator)
parameters['file'] = output_name
parameters['formatName'] = 'GeoTIFF-BigTiff'

node_id = 'Write'

source_node_id = 'Terrain-Correction'

mygraph.add_node(node_id, operator, parameters, source_node_id)

In [39]:
mygraph.view_graph()

<graph>
  <version>1.0</version>
  <node id="Read-S(0)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_SLC__1SDV_20170105T035027_20170105T035041_014691_017E74_5F66/S1A_IW_SLC__1SDV_20170105T035027_20170105T035041_014691_017E74_5F66.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="Read-S(1)">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <formatName/>
      <file>/workspace/data/S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110/S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110.SAFE/manifest.safe</file>
    </parameters>
  </node>
  <node id="SliceAssembly-S">
    <operator>SliceAssembly</operator>
    <sources>
      <sourceProduct refid="Read-S(0)"/>
      <sourceProduct.1 refid="Read-S(1)"/>
    </sources>
    <parameters class="com.bc.ceres.binding

In [33]:
mygraph.run()

Processing the graph
Process PID: 3762
Executing processing graph
Master: 05Jan2017
Slave: 05Jan2017 prep baseline: 0.0 temp baseline: 0.0
Slave: 17Jan2017 prep baseline: 30.903427 temp baseline: -12.000092

Master: 17Jan2017
Slave: 05Jan2017 prep baseline: -30.829674 temp baseline: 12.000092
Slave: 17Jan2017 prep baseline: 0.0 temp baseline: 0.0

Master: 05Jan2017
Slave: 05Jan2017 prep baseline: 0.0 temp baseline: 0.0
Slave: 17Jan2017 prep baseline: 28.903534 temp baseline: -12.000092

Master: 17Jan2017
Slave: 05Jan2017 prep baseline: -28.836931 temp baseline: 12.000092
Slave: 17Jan2017 prep baseline: 0.0 temp baseline: 0.0

Master: 05Jan2017
Slave: 05Jan2017 prep baseline: 0.0 temp baseline: 0.0
Slave: 17Jan2017 prep baseline: 27.083118 temp baseline: -12.000092

Master: 17Jan2017
Slave: 05Jan2017 prep baseline: -27.022404 temp baseline: 12.000092
Slave: 17Jan2017 prep baseline: 0.0 temp baseline: 0.0

..Java heap space
Java heap space
java.lang.NullPointerException
Java heap space
J

#### Results metadata

In [34]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % sar)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)

    polarisation_channels = etree.SubElement(acquisition, '{%s}polarisationChannels' % eop)
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)

#### Get the result WKT


In [35]:
src = gdal.Open(output_name + '.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt

#### Create the EOP XML metadata file

In [36]:
import json
eop_metadata = dict()

eop_metadata['wkt'] = result_wkt
eop_metadata['startdate'] = parser.parse(min(dates)).strftime('%Y-%m-%dT%H:%M:%S')
eop_metadata['enddate'] = parser.parse(max(dates)).strftime('%Y-%m-%dT%H:%M:%S')

eop_xml = output_name + '.xml'
with open(eop_xml, 'wb') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    file.write(json.dumps(eop_metadata))

#### Create the properties file for the reproducibility notebooks

In [37]:
for properties_file in ['result', 'stage-in']:

    if properties_file == 'result':
        title = 'Reproducibility notebook used for generating %s' % output_name
    else: 
        title = 'Reproducibility stage-in notebook for Sentinel-1 data for generating %s' % output_name
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (parser.parse(min(dates)).strftime('%Y-%m-%dT%H:%M:%S'), parser.parse(max(dates)).strftime('%Y-%m-%dT%H:%M:%S')))      
        file.write('geometry=%s' % (result_wkt))

### Compress the file

In [38]:
with open(output_name + '.tif', 'rb') as f_in, gzip.open(output_name + '.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
    
os.remove(output_name + '.tif')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.